# Calculate the correlations for each layer and create input RDMs

## Set up the environment

In [ ]:
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import seaborn
import pandas as pd

# os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Define the function to create filenames

In [ ]:
def getFileName(n_samples, name):
    return name \
        + "_{}_".format(n_samples) \
        + "_{}_".format(model_name) \
        + "_{}".format(layer_name)   \
        + ".npy"       

## Select the model and layer

In [ ]:
root_path = '/mnt/raid/ni/agnessa/rcnn-sat'
NR_OF_SAMPLES = 60

num_layers = 7
num_timepoints = 8
layer_names = []
for layer_idx in range(num_layers):
    for timepoint in range(num_timepoints):       
        layer_time = 'ReLU_Layer_{}_Time_{}'.format(layer_idx,timepoint)
        layer_names.append(layer_time)

model_name = '/model_02.11_2/'
path_dnn = root_path+model_name

# layer_name = layer_names[3]

## Define the correlation function

In [ ]:
# my version
def correlationd_matrix(batch_size): #(list_of_activations, n) ,array_activations
    file_name = os.path.join(path_dnn+'normalized_nopca/activations/','{}_activations.npy'.format(layer_name)) 
#     act = np.load(file_name,mmap_mode='r') #mmap is used to access a part of the file 
    act_all = np.load(file_name,mmap_mode='r')
    act = act_all[-NR_OF_SAMPLES:]
    print(np.argwhere(np.isnan(act)))
    correlationd = np.zeros((NR_OF_SAMPLES,NR_OF_SAMPLES))
    correlationd[:] = np.nan
    num_batches = int(NR_OF_SAMPLES / batch_size)
    total = sum(x+1 for x in range(num_batches)) #num comparisons to do
    index = 0
   
    for i in range(num_batches): 
        start_1 = batch_size*i
        end_1 = batch_size*(i+1)
        list_of_activations_1 = act[start_1:end_1,:]

        for j in range(i,num_batches): 
            index += 1
#             print("New Iteration: i = {0}, j = {1}; {2}/{3}".format(i,j,index,total))
            start_2 = batch_size*(j)
            end_2 = batch_size*(j+1)
            list_of_activations_2 = act[start_2:end_2,:]
            corr_activations = 1-np.corrcoef(list_of_activations_1,list_of_activations_2) 

            for x in range(corr_activations.shape[0]):
                for y in range(corr_activations.shape[1]):
                    if x < batch_size:
                        start_x = start_1
                    else: 
                        start_x = start_2-batch_size
                    if y < batch_size:
                        start_y = start_1
                    else:
                        start_y = start_2-batch_size                       
                    correlationd[x+start_x,y+start_y] = correlationd[y+start_y,x+start_x] = corr_activations[x,y]

    return(correlationd)

In [ ]:
for layer_name in layer_names:
    # layer_name = layer_names[0]
    print('Calculating the correlations for model: ',model_name,'and layer: ',layer_name)
    corr_matrix = correlationd_matrix(10) 
    path = os.path.join(path_dnn + 'normalized_nopca/Input_RDM/', '{}_Input_RDM.npy'.format(layer_name))
    print("Save Input RDM -> {}".format(path))
    np.save(path, np.array(corr_matrix)) 


In [ ]:
fig = plt.figure(figsize=(17,13))
ax = seaborn.heatmap(corr_matrix, cmap='rainbow', vmin=0.0, vmax=2.0)
# path_fig = os.path.join(ROOT_PATH + 'Input_RDM_plots', getFileName(NR_OF_SAMPLES,"Input_RDM_") + '.png')

In [ ]:
num_layers = 7
num_timepoints = 8
for layer in range(num_layers):
    directory = '/mnt/raid/ni/agnessa/rcnn-sat/model_02.11_2/normalized_nopca/'
    filename = 'ReLU_Layer_{}_Time_7_Input_RDM'.format(layer)
    mat = np.load(os.path.join(directory+'Input_RDM/','{}.npy'.format(filename)))
   
    fig = plt.figure(figsize=(17,13))
    ax = seaborn.heatmap(mat, cmap='rainbow', vmin=0.0, vmax=1.0)
    
    fig.savefig(os.path.join(directory+'Input_RDM_plots/','{}.png'.format(filename)))


## In case the load function does not work

In [ ]:
# #If the load function gives an error, do this
# np_load_old = np.load # modify the default parameters of np.load
# np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
# activations_shape = np.load(path)
# np.load = np_load_old